<a href="https://colab.research.google.com/github/mikofarin12/Amazon/blob/main/Copy_of_188c_hw3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#188C Homework 3: Finetuning CNN for Image Classification
In this homework, we will finetune a CNN model to classify whether people in the images are wearing masks.

Any changes won't be saved on this example notebook. Please edit on your own copy. 

Please submit the pdf version of your notebook(on the top left menu, click File --> Print to get the pdf).

##Step1: environment setup
In this tutorial, we will use PyTorch deep learning framework. For training acceleration, we can use free GPU offered in Google Colab. To change to GPU setting, click "Runtime --> change runtime type" and change acceleration type to GPU.

In [1]:
from __future__ import print_function 
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  1.8.0+cu101
Torchvision Version:  0.9.0+cu101


##Step 2: model input set up

Please upload the zip file https://drive.google.com/drive/folders/1J7zq8j03w1R4DzcIFiLIDuOgCxlOeugy?usp=sharing into your own Google drive and put it in a folder called mask_classification

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
from zipfile import ZipFile
with ZipFile('/content/gdrive/MyDrive/mask_classification/mask_image_set.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()

In [8]:
# path to the ImageFolder
data_dir = './mask_image_set'

# Number of classes in the dataset
num_classes = 2

# Batch size for training (change depending on how much memory you have)
batch_size = 64

# Number of epochs to train for 
num_epochs = 15

# When False, we finetune the whole model, when True we only update the reshaped layer params
feature_extract = True

# image size for the network input
input_size = 224

In PyTorch, data is organized using DataLoader and Dataset modules

Datasets: the abstract structure that organize all the images and labels

Dataloader: the generator to yeild data batch for model training at each step

In [35]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Create training and validation datasets
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'val']}

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


##Step 3: model initialization
There are lots of deep models with hundreds of layers trained on Imagenet, a large dataset including images of 1000 classes. We consider models trained on this large dataset have already gained plenty of visual knowledge, therefore, after we use our own data to finetune model, hopefully the model will learn to deal with our new task combining knowledge gained from task-specific new data and its previous visual knowledge trained from Imagenet.

To initialize the model, we can take advantage of TorchVision, a package saving plenty of deep model parameters. As mentioned above, the model is learned to classify 1000 classes. Here we only want to classify two classes, therefore, after downloading the model, we will change the dimension of the last layer to two.

In [36]:
# if feature_extract=True, this function will freeze all layers except the last layer
def set_parameter_requires_grad(model, feature_extract):
    if feature_extract:
        for param in model.parameters():
            param.requires_grad = False

In [37]:
model_ft = models.vgg11_bn(pretrained=True) # pretrained=True will initialize the model with parameters learned from Imagenet
set_parameter_requires_grad(model_ft, feature_extract)
num_ftrs = model_ft.classifier[6].in_features
model_ft.classifier[6] = nn.Linear(num_ftrs,2  ) # change the dimension of the last layer to two
model_ft.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU(inplace=True)
    (14): MaxPool2d(ke

##Step 4: optimization tools

In deep learning, we use loss metrics to evalute how close between the predicted label and the grouth truth. A smaller loss means a better performance. 

To minimize the loss at each time step, we will use the optimizer to compute the gradients and backpropagate through the network.

Here we will use sotochastic gradient descent as our optimizer and cross entropy as our loss metric.

In [12]:
# you can play with lr to find the best accuracy
optimizer = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss() 

##Step 5: training

In [13]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25):
    since = time.time()

    val_acc_history = []
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [14]:
model_ft = train_model(model_ft, dataloaders_dict, criterion, optimizer, num_epochs)

Epoch 0/14
----------


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


train Loss: 0.6895 Acc: 0.5633
val Loss: 0.5533 Acc: 0.7350

Epoch 1/14
----------
train Loss: 0.4932 Acc: 0.7750
val Loss: 0.4187 Acc: 0.8200

Epoch 2/14
----------
train Loss: 0.4166 Acc: 0.8050
val Loss: 0.3716 Acc: 0.8350

Epoch 3/14
----------
train Loss: 0.3750 Acc: 0.8517
val Loss: 0.3483 Acc: 0.8650

Epoch 4/14
----------
train Loss: 0.3457 Acc: 0.8550
val Loss: 0.3324 Acc: 0.8700

Epoch 5/14
----------
train Loss: 0.3413 Acc: 0.8417
val Loss: 0.3173 Acc: 0.8800

Epoch 6/14
----------
train Loss: 0.3274 Acc: 0.8600
val Loss: 0.3040 Acc: 0.8800

Epoch 7/14
----------
train Loss: 0.3289 Acc: 0.8667
val Loss: 0.3029 Acc: 0.8700

Epoch 8/14
----------
train Loss: 0.3119 Acc: 0.8717
val Loss: 0.2958 Acc: 0.8800

Epoch 9/14
----------
train Loss: 0.3090 Acc: 0.8700
val Loss: 0.2908 Acc: 0.8800

Epoch 10/14
----------
train Loss: 0.3178 Acc: 0.8550
val Loss: 0.2892 Acc: 0.8800

Epoch 11/14
----------
train Loss: 0.3324 Acc: 0.8583
val Loss: 0.2848 Acc: 0.8850

Epoch 12/14
----------
t

In [56]:
# test the model using test data
# load test dataset and create test dataloader as in step 2
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

image_datasets_test = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'test']}
# Create training and validation dataloaders
dataloaders_dict_test = {x: torch.utils.data.DataLoader(image_datasets_test[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train','test']}

def test_model(model, dataloaders, criterion, optimizer, num_epochs=25):
    since = time.time()

    val_acc_history = []
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to test mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'test':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best test Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

model_ft = test_model(model_ft, dataloaders_dict_test, criterion, optimizer, num_epochs)



Epoch 0/14
----------


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


train Loss: 0.7388 Acc: 0.4850
test Loss: 0.7071 Acc: 0.4800

Epoch 1/14
----------
train Loss: 0.7255 Acc: 0.5167
test Loss: 0.7067 Acc: 0.4800

Epoch 2/14
----------
train Loss: 0.7350 Acc: 0.4917
test Loss: 0.7063 Acc: 0.4750

Epoch 3/14
----------
train Loss: 0.7337 Acc: 0.4883
test Loss: 0.7079 Acc: 0.4700

Epoch 4/14
----------
train Loss: 0.7348 Acc: 0.4967
test Loss: 0.7085 Acc: 0.4750

Epoch 5/14
----------
train Loss: 0.7375 Acc: 0.4717
test Loss: 0.7078 Acc: 0.4700

Epoch 6/14
----------
train Loss: 0.7370 Acc: 0.4700
test Loss: 0.7063 Acc: 0.4700

Epoch 7/14
----------
train Loss: 0.7290 Acc: 0.4967
test Loss: 0.7058 Acc: 0.4750

Epoch 8/14
----------
train Loss: 0.7350 Acc: 0.4633
test Loss: 0.7072 Acc: 0.4750

Epoch 9/14
----------
train Loss: 0.7360 Acc: 0.5083
test Loss: 0.7068 Acc: 0.4750

Epoch 10/14
----------
train Loss: 0.7313 Acc: 0.4900
test Loss: 0.7074 Acc: 0.4700

Epoch 11/14
----------
train Loss: 0.7358 Acc: 0.4600
test Loss: 0.7077 Acc: 0.4700

Epoch 12/14
